# !Optional! delete all

MATCH (n)
OPTIONAL MATCH (n)-[r]-()
DELETE n, r;

# Load data into Neo4j

In [3]:
import pandas as pd
from neo4j.v1 import GraphDatabase, basic_auth

## initalize Neo4j driver

set host,user,passw 

In [4]:
host = 'bolt://localhost'
user = 'neo4j'
passw = 'admin'

In [5]:
driver = GraphDatabase.driver(host, auth=basic_auth(user, passw))
db_session = driver.session()

## load data as dataframe

set the directory having data.xlsx and *_hotel_users.xlsx files

In [ ]:
file_dir = '/usr/share/neo4j/import/'

In [43]:
data_df = pd.read_excel(file_dir + 'Data.xlsx', sheetname='Overview')


In [44]:
data_df.columns

Index(['city_id', 'city_name', 'forum_content', 'hotels_content',
       'hotels_quantity', 'restaurants_content', 'restaurants_quantity',
       'things to do_content', 'things to do_quantity',
       'vacation rentals_content', 'vacation rentals_quantity'],
      dtype='object')

## load cities into neo4j from dataframe

In [45]:
create_cypher = 'CREATE (city:City {{city_id:"{city_id}",city_name:"{city_name}", forum_content:{forum_content}, \
hotels_content:{hotels_content}, hotels_quantity:{hotels_quantity}, restaurants_content:{restaurants_content}, \
things_to_do_content:{things_to_do_content}, things_to_do_quantity:{things_to_do_quantity}, \
vacation_rentals_content:{vacation_rentals_content}, vacation_rentals_content:{vacation_rentals_content}}})'

for i,row in data_df.iterrows():
    tmp_cypher = create_cypher.format(city_id=row['city_id'],city_name=row['city_name'],forum_content=row['forum_content'],\
                              hotels_content=row['hotels_content'], hotels_quantity=row['hotels_quantity'], \
                              restaurants_content=row['restaurants_content'],\
                              restaurants_quantity=row['restaurants_quantity'],\
                              things_to_do_content=row['things to do_content'],\
                              things_to_do_quantity=row['things to do_quantity'],\
                              vacation_rentals_content=row['vacation rentals_content'],\
                              vacationrentals_quantity=row['vacation rentals_quantity'])
    db_session.run(tmp_cypher)

## iterate over files

In [46]:
relevant_cities = ['g274919','g274897','g274904','g274916',
                   'g274921', 'g776248']
#veszprem_hotels = 'g274919'

In [47]:
for city in relevant_cities:
    hotels_df = pd.read_excel(file_dir + city + '.xlsx', sheetname='Hotels')
    hotel_city_link_cypher = 'MERGE (h :Hotel {{name:"{hotel_name}", ratings: toFloat({ratings}),category:{category}\
    ,reviews:{reviews}}}) MERGE (c :City {{city_id:"{city_id}"}}) MERGE (h)-[:IS_IN]->(c)'
    for i, row in hotels_df.iterrows():
        tmp_cypher = hotel_city_link_cypher.format(hotel_name=row['name'],ratings=row['ratings'], \
                                                  category=row['category'], reviews=row['reviews'],
                                                  city_id=city)
        db_session.run(tmp_cypher)
        
    hotel_users_df = pd.read_excel(file_dir + city + '_hotel_users.xlsx')
    user_hotel_link_cypher = 'MERGE (u :User {{user:"{user}"}}) \
    MERGE (h :Hotel {{name:"{hotel_id}"}}) MERGE (u)-[:RATED {{rating: toFloat({rating})}}]->(h)'

    for i, row in hotel_users_df.iterrows():
        tmp_cypher = user_hotel_link_cypher.format(user=row['user'],hotel_id=row['hotel_id'],rating=row['rating'])
        db_session.run(tmp_cypher)